In [1]:
# -*- coding:utf-8 -*-
import sys, os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

/Users/sunchao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/sunchao/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
os.chdir("/Users/sunchao/Desktop/Guo/")

In [3]:
train_data = pd.read_csv("./X_train.csv", header=0)
feature_columns = ["MYCT","MMIN","MMAX","CACH","CHMIN","CHMAX"]
# remove id columns
train_data = train_data.iloc[:,1:]
# remove null columns
train_data.drop(['Unnamed: 7'],axis=1, inplace=True)

In [4]:
# split the train data of train-data and validation-data
X_train,X_val,y_train,y_val = train_test_split(train_data[feature_columns],train_data.PRP,train_size=0.9)

X_train = np.asarray(X_train)
X_train = scale(X_train)

X_val = np.asarray(X_val)
X_val = scale(X_val)

y_train = np.asarray(y_train).reshape((-1,1))
# y_train = scale(y_train)

y_val = np.asarray(y_val).reshape((-1,1))
# y_val = scale(y_val)

/Users/sunchao/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Users/sunchao/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [5]:
print("自变量---源数据:",train_data[feature_columns].shape, "；  训练集:",X_train.shape, "；  验证集:",X_val.shape)
print("因变量---源数据:",train_data.PRP.shape, "；  训练集:",y_train.shape, "；  验证集:",y_val.shape)

自变量---源数据: (168, 6) ；  训练集: (151, 6) ；  验证集: (17, 6)
因变量---源数据: (168,) ；  训练集: (151, 1) ；  验证集: (17, 1)


In [6]:
# define placeholder of neural network
xs = tf.placeholder(shape = [None, X_train.shape[1]], dtype=tf.float32, name="inputs")
ys = tf.placeholder(shape = [None, 1], dtype=tf.float32, name="y_true")
keep_prob_s = tf.placeholder(dtype=tf.float32)

In [7]:
# define the function of build neural network
def add_layer(inputs, input_size, output_size, activation_function=None):
    with tf.variable_scope("Weights"):
        Weights = tf.Variable(tf.random_normal(shape=[input_size,output_size]), name="weights")
    with tf.variable_scope("biases"):
        biases = tf.Variable(tf.zeros(shape=[1, output_size]) + 0.1, name="biases")
    with tf.name_scope("Wx_plus_b"):
        Wx_plus_b = tf.matmul(inputs, Weights) + biases
    with tf.name_scope("dropout"):
        Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob=keep_prob_s)
    if activation_function is None:
        return Wx_plus_b
    else:
        return activation_function(Wx_plus_b)

In [9]:
# Defining network structure
with tf.name_scope("input_layer"):
    input_layer = add_layer(xs, 6, 10, activation_function=tf.nn.relu)
    
with tf.name_scope("hidder_layer"):
    h_layer = add_layer(input_layer, 10, 10, activation_function=tf.nn.relu)
    
with tf.name_scope("prediction"):
    pred = add_layer(h_layer, 10, 1)

with tf.name_scope("Loss"):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys-pred),reduction_indices=[1]))  # MSE Error
with tf.name_scope("train"):
    train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [9]:
# parameter
keep_prob=1  # 防止过拟合，取值一般在0.5到0.8。我这里是1，没有做过拟合处理
ITER =5000  # 训练次数

In [10]:
def fit(X,y,n, keep_prob):
    init = tf.global_variables_initializer()
    feed_dict = {xs:X, ys:y, keep_prob_s:keep_prob}
    with tf.Session() as sess:

        saver = tf.train.Saver(tf.global_variables(), max_to_keep=15)
        merged = tf.summary.merge_all()
        writer = tf.summary.FileWriter(logdir="nn_log", graph=sess.graph)  #写tensorbord
        sess.run(init)
        for i in range(n):
            _loss, _ = sess.run([loss,train_op], feed_dict=feed_dict)
            
            if i % 100 == 0:
                print("epoch:%d\tloss:%.5f" % (i, _loss))
                y_pred = sess.run(pred, feed_dict = feed_dict)
                rs = sess.run(merged, feed_dict=feed_dict)
                writer.add_summary(summary=rs, global_step=i)  #写tensorbord
                saver.save(sess=sess, save_path="nn_model/nn.model", global_step=i) # 保存模型
        saver.save(sess=sess, save_path="nn_model/nn.model", global_step=n)  # 保存模型 

In [12]:
fit(X=X_train,y=y_train,n=10000,keep_prob=1)

epoch:0	loss:42913.12891


TypeError: Fetch argument None has invalid type <class 'NoneType'>